# Deep-Live-Cam on Colab T4 GPU
High-performance face swap with GPU acceleration

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
# Install system dependencies
!apt update -qq
!apt install -y ffmpeg

# Clone repository
!git clone https://github.com/Mayank-kanojiya/liveswap.git
%cd liveswap

In [ ]:
# Install GPU-optimized dependencies
!pip install -q opencv-python onnxruntime-gpu torch torchvision
!pip install -q customtkinter pillow psutil opennsfw2 protobuf cv2-enumerate-cameras
!pip install -q insightface
!pip install -q git+https://github.com/xinntao/BasicSR.git@master
!pip install -q git+https://github.com/TencentARC/GFPGAN.git@master

In [ ]:
# Download models
import os
os.makedirs('models', exist_ok=True)

!wget -q -O models/GFPGANv1.4.pth https://huggingface.co/hacksider/deep-live-cam/resolve/main/GFPGANv1.4.pth
!wget -q -O models/inswapper_128.onnx https://huggingface.co/hacksider/deep-live-cam/resolve/main/inswapper_128_fp16.onnx

In [ ]:
# GPU-optimized face swap
import cv2
import sys
import os
sys.path.append('.')

# Initialize with GPU support
import modules.globals
modules.globals.headless = True
modules.globals.execution_providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
modules.globals.frame_processors = ['face_swapper']

from modules.processors.frame.face_swapper import get_face_swapper, process_frame
from modules.face_analyser import get_one_face
from modules.utilities import is_video, extract_frames, create_video, restore_audio, get_temp_frame_paths, create_temp, clean_temp

# Initialize face swapper with GPU
face_swapper = get_face_swapper()
print("Face swapper initialized with GPU support")

def face_swap_gpu(source_path, target_path, output_path, mouth_mask=False, many_faces=False):
    # Set options
    modules.globals.mouth_mask = mouth_mask
    modules.globals.many_faces = many_faces
    modules.globals.source_path = source_path
    modules.globals.target_path = target_path
    modules.globals.output_path = output_path
    
    # Read source
    source_img = cv2.imread(source_path)
    source_face = get_one_face(source_img)
    
    if is_video(target_path):
        print("Processing video with GPU acceleration...")
        create_temp(target_path)
        extract_frames(target_path)
        
        temp_frame_paths = get_temp_frame_paths(target_path)
        
        for i, frame_path in enumerate(temp_frame_paths):
            frame = cv2.imread(frame_path)
            result_frame = process_frame(source_face, frame)
            cv2.imwrite(frame_path, result_frame)
            
            if i % 30 == 0:
                print(f"GPU processed {i+1}/{len(temp_frame_paths)} frames")
        
        create_video(target_path)
        restore_audio(target_path, output_path)
        clean_temp(target_path)
        
    else:
        print("Processing image with GPU acceleration...")
        target_img = cv2.imread(target_path)
        result_frame = process_frame(source_face, target_img)
        cv2.imwrite(output_path, result_frame)
    
    print(f"GPU face swap completed: {output_path}")

# Upload your files and run
# face_swap_gpu("source.jpg", "target.mp4", "result.mp4", mouth_mask=True)

In [ ]:
# Example usage - replace with your file paths
source_image = "/content/source_face.jpg"  # Upload your source image
target_media = "/content/target_video.mp4"  # Upload your target video/image
output_file = "/content/result.mp4"

# Run face swap with GPU acceleration
try:
    face_swap_gpu(source_image, target_media, output_file, mouth_mask=True)
    
    # Download result
    from google.colab import files
    files.download(output_file)
    
except Exception as e:
    print(f"Error: {e}")